# House Price Predictor 

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data_path = "./kc_house_data.csv"
data = pd.read_csv(data_path)
data = data.dropna(axis=0)
print(data.columns)

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')


## Set y and X values

In [3]:
y = data["price"].astype(np.float32)
data_prediction_features = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built"]
X = data[data_prediction_features].astype(np.float32)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Normalize the data

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Create Model

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4), # add various dropout layers to prevent overfitting
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression task, so no activation function (linear activation)
])

2023-08-15 10:13:48.077818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
loss_fn = tf.keras.losses.MeanSquaredError()
# ues adam optimizer function for dynamically updating gradient descent.
optimizerFunction = tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    beta_1=0.9,
    beta_2=0.995,
    epsilon=5e-06,
    amsgrad=True,
    name='Adam',
)
model.compile(optimizer=optimizerFunction,
              loss=loss_fn)

## Start Training

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=30) # add a callback to prevent overfitting (but be very patient)
model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.1, callbacks=[callback])


Epoch 1/150
487/487 [==============================] - 2s 3ms/step - loss: 258208055296.0000 - val_loss: 61297631232.0000
Epoch 2/150
487/487 [==============================] - 1s 3ms/step - loss: 61170601984.0000 - val_loss: 47409180672.0000
Epoch 3/150
487/487 [==============================] - 1s 3ms/step - loss: 52235173888.0000 - val_loss: 41103679488.0000
Epoch 4/150
487/487 [==============================] - 1s 2ms/step - loss: 47441559552.0000 - val_loss: 38896726016.0000
Epoch 5/150
487/487 [==============================] - 1s 2ms/step - loss: 47226023936.0000 - val_loss: 38121553920.0000
Epoch 6/150
487/487 [==============================] - 1s 2ms/step - loss: 46381244416.0000 - val_loss: 37458554880.0000
Epoch 7/150
487/487 [==============================] - 1s 2ms/step - loss: 45331365888.0000 - val_loss: 37547237376.0000
Epoch 8/150
487/487 [==============================] - 1s 2ms/step - loss: 45238206464.0000 - val_loss: 37322747904.0000
Epoch 9/150
487/487 [==========

## Evaluate Model

In [9]:
loss = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)

136/136 [==============================] - 0s 2ms/step - loss: 42128068608.0000
Test loss: 42128068608.0


## Predict the price of user input.
Enter in the various attributes into the user_input array. The order is the order of the X data input.

In [10]:
user_input = [3, 1.75, 3000, 30000, 2, 0, 0, 10, 7, 3000, 0, 2015]

# Normalize the user input using the same scaler
user_input_scaled = scaler.transform(np.array(user_input).reshape(1, -1))

# Predict the price using the trained model
predicted_price = model.predict(user_input_scaled)[0][0]

print(f"Estimated price: ${predicted_price:.2f}")

1/1 [==============================] - 0s 119ms/step
Estimated price: $975239.56


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
